In [ ]:
cd /content/drive/MyDrive/Thesis

/content/drive/MyDrive/Thesis


In [ ]:
!pip install -r requirements.txt

     |████████████████████████████████| 45 kB 1.9 MB/s 
     |████████████████████████████████| 311 kB 9.3 MB/s 
     |████████████████████████████████| 3.5 MB 63.9 MB/s 
     |████████████████████████████████| 312 kB 51.8 MB/s 
     |████████████████████████████████| 311 kB 57.7 MB/s 
     |████████████████████████████████| 311 kB 56.4 MB/s 
     |████████████████████████████████| 306 kB 50.9 MB/s 
     |████████████████████████████████| 298 kB 53.4 MB/s 
     |████████████████████████████████| 298 kB 53.8 MB/s 
     |████████████████████████████████| 290 kB 66.6 MB/s 
     |████████████████████████████████| 290 kB 62.5 MB/s 
     |████████████████████████████████| 290 kB 63.2 MB/s 
     |████████████████████████████████| 287 kB 58.8 MB/s 
     |████████████████████████████████| 287 kB 57.9 MB/s 
     |████████████████████████████████| 287 kB 60.7 MB/s 
     |████████████████████████████████| 285 kB 63.7 MB/s 
     |████████████████████████████████| 270 kB 50.3 MB/s 
     |███████████

In [ ]:
model_checkpoint = "gpt2"


In [ ]:
!pip install tqdm -U


     |████████████████████████████████| 76 kB 2.7 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


In [ ]:
from datasets import load_metric


In [ ]:
from transformers import GPT2LMHeadModel, DataCollatorForLanguageModeling, TrainingArguments, Trainer

model = GPT2LMHeadModel.from_pretrained(model_checkpoint)

In [ ]:
from transformers import GPT2Tokenizer
    
def load_tokenizer():
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    special_tokens = {'bos_token':'<|startoftext|>','eos_token':'<|endoftext|>','pad_token':'<pad>','additional_special_tokens':['<TITLE>']} 
    tokenizer.add_special_tokens(special_tokens)
    return tokenizer

tokenizer = load_tokenizer()
model.resize_token_embeddings(len(tokenizer))

Embedding(50260, 768)

In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Thesis/data/1024_length_data/train_pairs.csv", index_col=0)
df

,title,abstract,title_length,abstract_length
0,Natural Image Bases to Represent Neuroimaging ...,Visual inspection of neuroimagery is susceptib...,7,132
1,Sluice Resolution without Hand-Crafted Feature...,Sluice resolution in English is the problem of...,9,110
2,Learning Translation Models from Monolingual C...,Translation models often fail to generate good...,7,152
3,Sentiment Adaptive End-to-End Dialog Systems,End-to-end learning framework is useful for bu...,5,119
4,User-Friendly Text Prediction For Translators,Text prediction is a form of interactive machi...,5,134
...,...,...,...,...
21437,"Arabic Tokenization, Part-of-Speech Tagging an...",We present an approach to using a morphologica...,11,58
21438,Using Semantically Motivated Estimates to Help...,Research into the automatic acquisition of sub...,8,530
21439,A Mathematical Exploration of Why Language Mod...,"Autoregressive language models, pretrained usi...",11,194
21440,Do You Know That Florence Is Packed with Visit...,"When a speaker, Mary, asks ""Do you know that F...",15,174


In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
df.to_csv("/content/drive/MyDrive/Thesis/data/1024_length_data/1024_characters_pairs.csv")

In [ ]:
df = df.drop(columns=["title_length", "abstract_length", "token_len"])

In [ ]:
df_train = df[:11864]
df_valid = df[11864:11964]

In [ ]:
def add_input(df):
  inputs = []
  for a,t in zip(df.abstract.to_list(), df.title.to_list()):
    input = a + "<TITLE>" + t + "<|endoftext|>"
    inputs.append(input)
  df["input"] = inputs
  return df

In [ ]:
df_train = add_input(df_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
df_valid = add_input(df_valid)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
df_train = df_train.drop(columns=["title", "abstract"])
df_valid = df_valid.drop(columns=["title", "abstract"])

In [ ]:
from datasets import Dataset
from datasets import load_dataset, load_metric
train_dataset = Dataset.from_pandas(df_train)
valid_dataset = Dataset.from_pandas(df_valid)
metric = load_metric("rouge")


Downloading:   0%|          | 0.00/2.17k [00:00<?, ?B/s]

In [ ]:
max_input_length = 620
def preprocess_function(examples):
    model_inputs = tokenizer(examples["input"], padding="max_length", max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    labels = model_inputs

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [ ]:
train_dataset = train_dataset.map(preprocess_function, batched=True)
valid_dataset = valid_dataset.map(preprocess_function, batched=True)


  0%|          | 0/12 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
from operator import indexOf
def get_max_len(train_dataset, valid_dataset, max_global_input_len):
  for i in train_dataset:
    t = indexOf(i["input_ids"], tokenizer.eos_token_id) + 1
    if t > max_global_input_len:
      max_global_input_len = t

  for i in valid_dataset:
    t = indexOf(i["input_ids"], tokenizer.eos_token_id) + 1
    if t > max_global_input_len:
      max_global_input_len = t
  return max_global_input_len

max_global_input_len = get_max_len(train_dataset, valid_dataset, 0)

In [ ]:
batch_size = 2
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    "output/gpt2/" + f"{model_name}-finetuned-lm_al_paper",
    evaluation_strategy = "epoch",
    learning_rate=3e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    fp16=True,
    push_to_hub=False,
    gradient_accumulation_steps=8,
    save_steps = 200,
    logging_steps = 185,
)

In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)





```
Examples:
    >>> predictions = ["hello there", "general kenobi"]
    >>> references = ["hello there", "general kenobi"]
    >>> bertscore = datasets.load_metric("bertscore")
    >>> results = bertscore.compute(predictions=predictions, references=references, lang="en")
    >>> print([round(v, 2) for v in results["f1"]])
    [1.0, 1.0]
  
```



In [ ]:
import nltk
import numpy as np
nltk.download('punkt')
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

Using amp fp16 backend


In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: input.
***** Running training *****
  Num examples = 11864
  Num Epochs = 3
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 8
  Total optimization steps = 2223


Epoch,Training Loss,Validation Loss
0,3.169100,3.008055
1,2.846800,2.927294
2,2.624400,2.920357


Saving model checkpoint to output/gpt2/gpt2-finetuned-lm_al_paper/checkpoint-200
Configuration saved in output/gpt2/gpt2-finetuned-lm_al_paper/checkpoint-200/config.json
Model weights saved in output/gpt2/gpt2-finetuned-lm_al_paper/checkpoint-200/pytorch_model.bin
tokenizer config file saved in output/gpt2/gpt2-finetuned-lm_al_paper/checkpoint-200/tokenizer_config.json
Special tokens file saved in output/gpt2/gpt2-finetuned-lm_al_paper/checkpoint-200/special_tokens_map.json
added tokens file saved in output/gpt2/gpt2-finetuned-lm_al_paper/checkpoint-200/added_tokens.json
Saving model checkpoint to output/gpt2/gpt2-finetuned-lm_al_paper/checkpoint-400
Configuration saved in output/gpt2/gpt2-finetuned-lm_al_paper/checkpoint-400/config.json
Model weights saved in output/gpt2/gpt2-finetuned-lm_al_paper/checkpoint-400/pytorch_model.bin
tokenizer config file saved in output/gpt2/gpt2-finetuned-lm_al_paper/checkpoint-400/tokenizer_config.json
Special tokens file saved in output/gpt2/gpt2-fine

TrainOutput(global_step=2223, training_loss=3.166638523192755, metrics={'train_runtime': 4347.4786, 'train_samples_per_second': 8.187, 'train_steps_per_second': 0.511, 'total_flos': 1.125907365888e+16, 'train_loss': 3.166638523192755, 'epoch': 3.0})

In [ ]:
model.save_pretrained("./output/gpt2")


Configuration saved in ./output/gpt2/config.json
Model weights saved in ./output/gpt2/pytorch_model.bin


In [ ]:
import pandas as pd
test_samples = pd.read_csv("/content/drive/MyDrive/Thesis/data/1024_length_data/test_pairs.csv", index_col=0)
test_samples

,title,abstract,title_length,abstract_length
0,Learning Latent Semantic Annotations for Groun...,Previous work on grounded language learning di...,11,121
1,Partially Supervised Sense Disambiguation by L...,Supervised and semi-supervised sense disambigu...,13,140
2,Hawkes Processes for Continuous Time Sequence ...,Classification of temporal textual data sequen...,15,68
3,A Unified Single Scan Algorithm for Japanese B...,We describe an algorithm for Japanese analysis...,13,62
4,Generating Coherent Event Schemas at Scale,Chambers and Jurafsky (2009) demonstrated that...,6,127
...,...,...,...,...
5356,Bridging Information-Seeking Human Gaze and Ma...,"In this work, we analyze how human gaze during...",8,118
5357,Quantum-inspired Neural Network for Conversati...,We provide a novel perspective on conversation...,7,116
5358,The BQ Corpus: A Large-scale Domain-specific C...,This paper introduces the Bank Question (BQ) c...,13,174
5359,Doc2hash: Learning Discrete Latent variables f...,Learning to hash via generative model has beco...,8,131


In [ ]:
abstracts = test_samples.abstract.to_list()
titles = test_samples.title.to_list()

In [ ]:
def creat_eval_pairs(model, tokenizer, abstracts, titles):
  preds = []
  for abstract, title in zip(abstracts, titles):
    encoding = tokenizer(abstract + "<TITLE>", return_tensors = "pt", max_length=620)
    inputs = encoding["input_ids"].to("cuda")
    attention_masks = encoding["attention_mask"].to("cuda")
    title_ids = model.generate(
            input_ids = inputs,
            attention_mask = attention_masks,
            max_length = 1024,
            num_beams = 5,
            num_return_sequences = 5,
            repetition_penalty=2.0, 
            length_penalty=10.0,
            early_stopping = True,
            )
    result = []
    for g in title_ids:
      result.append(tokenizer.decode(g).split("<TITLE>")[1].split("<|endoftext|>")[0])
    s=""
    for t in result:
      s = s + "<TITLE>" + t
    preds.append(s)
    if len(preds) % 500 == 0:
      print("original title: ", title)
      print("generated title: ", preds[-1:])
  return preds, titles

In [ ]:
model.to("cuda")

In [ ]:
preds, titles = creat_eval_pairs(model, tokenizer, abstracts, titles)

In [ ]:
pred_target_pairs = pd.DataFrame(list(zip(preds, titles)), columns=['predictions', 'targets'])

In [ ]:
pred_target_pairs.to_csv("gpt2.csv")